In [1]:
import os
import torch
from utils.tools import check_all_exist
from monai.transforms import LoadImage, Compose, SaveImage
from monai.data import MetaTensor
from utils.cascade import (
    list_contour_slices,
    get_athero_status,
    fix_nearest_athero,
    get_mask_slice,
)
from tqdm import tqdm

root_dir = os.getenv("DATA_ROOT")

# COSMOS generate

In [4]:
input_dir = os.path.join(root_dir, "COSMOS", "train_data")
output_dir = os.path.join(root_dir, "COSMOS", "preprocessed", "mri_nii_raw")
mri_cases = [case for case in os.listdir(input_dir) if case.isdigit()]

image_loader = LoadImage(image_only=True, ensure_channel_first=True)
image_saver = SaveImage(output_dir, "image", resample=False, separate_folder=False, print_log=False)
mask_saver = SaveImage(output_dir, "mask", resample=False, separate_folder=False, print_log=False)

for case_file in tqdm(mri_cases):
    image_dir = os.path.join(output_dir, f"{case_file}_image.nii.gz")
    mask_dir = os.path.join(output_dir, f"{case_file}_mask.nii.gz")
    if not check_all_exist([image_dir, mask_dir]):  # check all files processed?
        dcm_image_dir = os.path.join(input_dir, case_file)
        image = image_loader(dcm_image_dir)
        C, H, W, D = image.shape
        new_mask = torch.zeros_like(image)
        athero_status = get_athero_status(input_dir, case_file)
        for art_i in ["L", "R"]:  # Read from given `CASCADE` format xml files
            annotated_slices = list_contour_slices(dcm_image_dir, art_i, case_file)
            if art_i in athero_status:
                athero_status[art_i] = fix_nearest_athero(annotated_slices, athero_status[art_i])
            else:
                athero_status[art_i] = {i:0 for i in annotated_slices}
            for i, anno_id in enumerate(annotated_slices):
                lumen_mask, wall_mask = get_mask_slice(dcm_image_dir, art_i, case_file, anno_id, H, W)
                wall_value = 3 if athero_status[art_i][anno_id] else 2  # 2 if heathy, 3 if diseased
                lumen_mask, wall_mask = lumen_mask, wall_mask * wall_value
                new_mask[..., anno_id] = torch.tensor(lumen_mask + wall_mask)
        new_mask = MetaTensor(new_mask, meta=image.meta)
        image_saver(image)
        mask_saver(new_mask)

100%|██████████| 50/50 [06:16<00:00,  7.53s/it]


## INTERP

In [3]:
from monai.transforms import LoadImage, SaveImage
import os 
import torch
from pathlib import Path
from tqdm import tqdm

dataset = 'COSMOS'
root_dir = os.getenv('DATA_ROOT')
raw_dir = os.path.join(root_dir, dataset, "train_data")
input_dir = os.path.join(root_dir, dataset, "preprocessed", "mri_nii_raw")
output_dir = os.path.join(root_dir, dataset, "preprocessed", "mri_nii_raw")
if dataset == "COSMOS":
    mri_cases = [case for case in os.listdir(raw_dir) if case.isdigit()]
else:
    mri_cases = [case for case in os.listdir(raw_dir) if case.startswith("0_P")]

mask_saver = SaveImage(output_dir, "interp", resample=False, separate_folder=False, print_log=False)
mask_loader = LoadImage(image_only=True, ensure_channel_first=True)

for case_file in tqdm(mri_cases):
    mask = mask_loader(os.path.join(input_dir, f"{case_file}_mask.nii.gz"))
    if os.path.exists(os.path.join(output_dir, f"{case_file}_interp.nii.gz")):
        continue
    C, H, W, D = mask.shape
    mask_L, mask_R = mask[:, :H//2], mask[:, H//2:]
    for mask_part, side in zip([mask_L, mask_R], ['L', 'R']):
        anno_id = torch.where(mask_part > 0)[-1].unique()
        if anno_id.shape[0] > 0:
            anno_id = anno_id
            unanno_slices = list()
            l_min, l_max = anno_id.min(), anno_id.max() + 1
            for s in range(l_min, l_max):
                if s not in anno_id:
                    unanno_slices.append(s)
            for s in unanno_slices:
                mask_part[..., s] = mask_part[..., min(anno_id, key=lambda x: abs(x - s))]
        else:
            print(f"{case_file}_{side}_image.nii.gz")
    mask[:, :H//2], mask[:, H//2:] = mask_L, mask_R
    mask.meta['filename_or_obj'] = f"{case_file}.nii.gz"
    mask_saver(mask)

 34%|███▍      | 17/50 [00:34<01:05,  1.99s/it]

28_R_image.nii.gz


 36%|███▌      | 18/50 [00:36<01:02,  1.95s/it]

29_R_image.nii.gz


 64%|██████▍   | 32/50 [01:06<00:36,  2.03s/it]

...

 86%|████████▌ | 43/50 [01:30<00:15,  2.15s/it]

52_R_image.nii.gz


 94%|█████████▍| 47/50 [01:38<00:06,  2.08s/it]

7_L_image.nii.gz


100%|██████████| 50/50 [01:46<00:00,  2.12s/it]
